In [1]:
from sktime.datasets import load_arrow_head  # univariate dataset
X_train_orig, y_train_orig = load_arrow_head(split="test", return_X_y=True)
X_test_orig, y_test_orig = load_arrow_head(split="train", return_X_y=True)

In [2]:
X_train_orig

,dim_0
0,0 -1.907777 1 -1.904890 2 -1.88856...
1,0 -1.829855 1 -1.808497 2 -1.79577...
2,0 -1.801563 1 -1.775734 2 -1.72848...
3,0 -1.885718 1 -1.855170 2 -1.84519...
4,0 -1.959051 1 -1.974930 2 -1.97141...
...,...
170,0 -1.625142 1 -1.622988 2 -1.62606...
171,0 -1.657757 1 -1.664673 2 -1.63264...
172,0 -1.603279 1 -1.587365 2 -1.57740...
173,0 -1.739020 1 -1.741534 2 -1.73286...


In [3]:
from sktime.classification.shapelet_based import ShapeletTransformClassifier

model = ShapeletTransformClassifier()
model.fit(X_train_orig, y_train_orig)

model.score(X_test_orig, y_test_orig)

0.8611111111111112

In [4]:
X_train_orig.iloc[0][0]

model.predict_proba(X_train_orig.iloc[0:1, :]).reshape(-1)

array([1., 0., 0.])

In [ ]:
from pyreal.explainers.time.occlusion_importance import OcclusionImportance

explainer = OcclusionImportance(model, X_train_orig, window_size=3, classes=['0', '1', '2'])
explainer.fit()
explanation = explainer.produce(X_train_orig.iloc[0][0])

251
